## XỬ LÍ - LÀM SẠCH DỮ LIỆU


#### I. Mô tả dữ liệu 

In [2]:
import pandas as pd

In [3]:
# Đọc dữ liệu từ file csv
stroke_preditition_df = pd.read_csv('healthcare-dataset-stroke-data.csv')
stroke_preditition_df.head(10)

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,9046,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,51676,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,NaN,never smoked,1
2,31112,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
3,60182,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
4,1665,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1
5,56669,Male,81.0,0,0,Yes,Private,Urban,186.21,29.0,formerly smoked,1
6,53882,Male,74.0,1,1,Yes,Private,Rural,70.09,27.4,never smoked,1
7,10434,Female,69.0,0,0,No,Private,Urban,94.39,22.8,never smoked,1
8,27419,Female,59.0,0,0,Yes,Private,Rural,76.15,NaN,Unknown,1
9,60491,Female,78.0,0,0,Yes,Private,Urban,58.57,24.2,Unknown,1


In [ ]:
# Kiểm tra thông tin sơ bộ của Data Frame
stroke_preditition_df.info()

Dữ liệu được lấy từ bộ dữ liệu chăm sóc sức khỏe (Healthcare Dataset - Stroke Data), gồm nhiều thông tin liên quan đến hồ sơ y tế của bệnh nhân.
Một số cột quan trọng trong tập dữ liệu gồm:

- id: Mã định danh bệnh nhân (không dùng trong dự đoán).
- gender: Giới tính.
- age: Tuổi.
- hypertension: Tình trạng huyết áp cao.
- heart_disease: Bệnh tim.
- ever_married: Tình trạng hôn nhân.
- work_type: Loại công việc.
- Residence_type: Khu vực sống (nông thôn/thành thị).
- avg_glucose_level: Mức đường huyết trung bình.
- bmi: Chỉ số khối cơ thể.
- smoking_status: Tình trạng hút thuốc.
stroke: Cột mục tiêu (1 nếu có đột quỵ, 0 nếu không).

#### II. Thực hiện tiền xử lí

1. Xử lý Dữ liệu thiếu (Handling Missing Values):

In [5]:

# Xử lý giá trị thiếu ở cột BMI
stroke_preditition_df['bmi'].fillna(stroke_preditition_df['bmi'].median(), inplace=True)



- Vì BMI là dữ liệu số, thường phân bố lệch nhẹ, nên median (trung vị) sẽ an toàn hơn mean (trung bình) khi có outlier.

2. Mã hóa dữ liệu (Encoding Categorical Data):

In [6]:
# Các cột cần mã hóa
categorical_cols = ['gender', 'ever_married', 'work_type', 'Residence_type', 'smoking_status']

# Tạo DataFrame chứa các cột đã mã hóa
encoded = pd.get_dummies(stroke_preditition_df[categorical_cols], drop_first=True)

# Ghép lại vào stroke_preditition_df gốc, giữ nguyên thứ tự các cột ban đầu
stroke_preditition_df = pd.concat([stroke_preditition_df.drop(columns=categorical_cols), encoded], axis=1)


- Áp dụng phương pháp One-Hot Encoding để chuyển các giá trị phân loại thành các biến nhị phân (0 và 1). Mỗi giá trị trong một cột được mã hóa thành một cột mới đại diện riêng biệt.
Việc này giúp mô hình hiểu đúng bản chất của dữ liệu, tránh việc hiểu sai thứ tự giữa các giá trị (như khi dùng Label Encoding), đồng thời cải thiện khả năng học và dự đoán của mô hình.

3. Xử lý Dữ liệu không liên quan (Handling Irrelevant Features):

In [7]:
# Xóa cột id không liên quan
stroke_preditition_df = stroke_preditition_df.drop( columns= 'id', inplace = False)

- Trong bộ dữ liệu, cột id chỉ đóng vai trò là mã số định danh cho từng bản ghi (record). Nó không mang thông tin thực chất nào về đặc điểm của bệnh nhân hoặc kết quả bệnh lý, nên không có giá trị dự đoán trong mô hình học máy.

In [ ]:
stroke_preditition_df.head(10)